**1.导入和查验数据**

In [6]:
import pandas as pd

digits_train=pd.read_csv('/Users/scarlett/repository/projects/digits/optdigits.tra',header=None)
digits_test=pd.read_csv('/Users/scarlett/repository/projects/digits/optdigits.tes',header=None)
# header=None 表示第一行是数据而非文件第一行

print digits_train.shape,digits_test.shape
print digits_train.head()
print digits_train.info()

(3823, 65) (1797, 65)
   0   1   2   3   4   5   6   7   8   9  ...  55  56  57  58  59  60  61  62  \
0   0   1   6  15  12   1   0   0   0   7 ...   0   0   0   6  14   7   1   0   
1   0   0  10  16   6   0   0   0   0   7 ...   0   0   0  10  16  15   3   0   
2   0   0   8  15  16  13   0   0   0   1 ...   0   0   0   9  14   0   0   0   
3   0   0   0   3  11  16   0   0   0   0 ...   0   0   0   0   1  15   2   0   
4   0   0   5  14   4   0   0   0   0   0 ...   0   0   0   4  12  14   7   0   

   63  64  
0   0   0  
1   0   0  
2   0   7  
3   0   4  
4   0   6  

[5 rows x 65 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3823 entries, 0 to 3822
Data columns (total 65 columns):
0     3823 non-null int64
1     3823 non-null int64
2     3823 non-null int64
3     3823 non-null int64
4     3823 non-null int64
5     3823 non-null int64
6     3823 non-null int64
7     3823 non-null int64
8     3823 non-null int64
9     3823 non-null int64
10    3823 non-null int64
11 

**2.使用K-means算法识别手写体图像数据**

图像数据由8*8像素矩阵表示，64个像素维度；1个目标维度用来标记每个图像样本代表的数字类别

In [14]:
import numpy as np
import matplotlib.pyplot as plt

# 从训练集合测试集上都分离出64维度的像素特征和1维度的数字目标
X_train=digits_train[np.arange(64)]
y_train=digits_train[64]
X_test=digits_test[np.arange(64)]
y_test=digits_test[64]

from sklearn.cluster import KMeans

# 初始化模型，并设置聚类中心数量为10
kmeans=KMeans(n_clusters=10)
kmeans.fit(X_train)
# 逐条判断每个测试图像所属的聚类中心
y_pred=kmeans.predict(X_test)

**3.性能测评**

In [8]:
# 使用ARI进行K-means聚类性能评估
from sklearn import metrics
print metrics.adjusted_rand_score(y_test,y_pred)

0.659285217314107


In [ ]:
# 使用轮廓系数评估不同类簇数量的
from sklearn.cluster import KMeans

# 导入计算轮廓系数的模块
from sklearn.metrics import silhouette_score

# 分割出3*2=6个子图，并在1号子图作图；subplot(m,n,p)是将多个图画到一个平面上的工具,m行n列，p=1代表从左到右从上到下的第一个位置
plt.subplot(3,2,1)

# 初始化原始数据点
x1=np.array([1,2,3,1,5,6,5,5,6,7,8,9,7,9])
x2=np.array([1,3,2,2,8,6,7,6,7,1,2,1,1,3])
X=np.array(zip(x1,x2)).reshape(len(x1),2)

# 在1号子图做出原始数据点阵的分布
plt.xlim([0,10])
plt.ylim([0,10])
plt.title('Instance')
plt.scatter(x1,x2)

colors=['b','g','r','c','m','y','k','b']
markers=['o','s','D','v','^','p','*','+']

clusters=[2,3,4,5,8]
subplot_counter=1
sc_scores=[]

for t in clusters:
    subplot_counter += 1
    plt.subplot(3,2,subplot_counter)
    kmeans_model=KMeans(n_clusters=t).fit(X)
    
for i,l in enumerate(kmeans_model.labels_):
    plt.plot(x1[i],x2[i],color=colors[l],marker=markers[l],ls='None')
    
plt.xlim([0,10])
plt.ylim([0,10])
sc_score=silhouette_score(X,kmeans_model.labels_,metric='euclidean')
sc_scores.append(sc_score)

plt.title('K=%s,silhouette coefficient=%0.03f' %(t,sc_score))

plt.figure()
plt.plot(clusters,sc_scores,'*-')
plt.xlabel('Number of Clusters')
plt.ylabel('silhouette coefficient score')

plt.show()

由图可知，当聚类中心数量k=3时，轮廓系数最大；由轮廓系数与不同类簇数量的关系曲线可知，聚类中心数量为3也符合数据分布特点。

特点分析：

- K-means聚类模型采取的是迭代式算法
- 缺点：容易收敛到局部最优解；需要预先设定簇的数量

**肘部观察法示例**

In [17]:
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

# 使用均匀分布函数随机三个簇，每个簇周围10个数据样本
cluster1=np.random.uniform(0.5,1.5,(2,10))
cluster2=np.random.uniform(5.5,6.5,(2,10))
cluster3=np.random.uniform(10.5,11.5,(2,10))

# 绘制30个数据样本的分布图像
X=np.hstack((cluster1,cluster2,cluster3)).T
plt.scatter(X[:,0],X[:,1])
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

# 测试9种不同聚类中心数量下，每种情况的聚类质量
K=range(1,10)
meandistortions=[]

for k in K:
    kmeans=KMeans(n_clusters=k)
    kmeans.fit()
    meandistortions.append(sum(np.min(cdist(X,kmeans.cluster_centers_,'euclidean'),axis=1))/X.shape[0])

    
plt.plot(K,meandistortions,'bx-')
plt.xlabel('k')
plt.ylabel('Average Dispersion')
plt.title('Selecting k with the Elbow Method')
plt.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':


TypeError: fit() takes at least 2 arguments (1 given)